In [1]:
from langchain_ollama import ChatOllama

ollama_llm = ChatOllama(
    model="deepseek-v2",
    temperature=0.7,
    )


In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import os

_=load_dotenv(find_dotenv())
DEEPSEEK_API = os.getenv("DEEPSEEK_API")
BASE_URL = os.getenv("DEEPSEEK_URL")
MODEL_NAME = os.getenv("DEEPSEEK_MODEL")
llm = ChatOpenAI(api_key=DEEPSEEK_API, base_url=BASE_URL, model=MODEL_NAME)

In [3]:
llm.invoke("你好")

AIMessage(content='你好！欢迎使用人工智能助手。有什么我可以帮助你的吗？无论是问题解答、信息查询还是其他任何需求，请随时告诉我。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 4, 'total_tokens': 32, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 4}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_7e0991cad4', 'finish_reason': 'stop', 'logprobs': None}, id='run-7c3170fd-23fa-4c91-98a0-7f96548403b9-0', usage_metadata={'input_tokens': 4, 'output_tokens': 28, 'total_tokens': 32})

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# llm=llm.with_structured_output(Classification)
tagging_chain = tagging_prompt | llm

In [5]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
tagging_chain.invoke({"input": inp})

AIMessage(content="The passage provided does not contain any information related to a 'Classification' function or properties. It is a personal message expressing happiness about meeting someone and the expectation of becoming good friends. There is no technical or classification-related content to extract from this text.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 56, 'total_tokens': 107, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 56}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_7e0991cad4', 'finish_reason': 'stop', 'logprobs': None}, id='run-3bf9f83c-277a-4c1f-ba66-72f74dfad3e4-0', usage_metadata={'input_tokens': 56, 'output_tokens': 51, 'total_tokens': 107})

In [6]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

In [7]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [8]:
prompt_performance_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [9]:
from entity.extraction import PerformanceQuerySchema



In [10]:


runnable =prompt_performance_template | llm.with_structured_output(schema=PerformanceQuerySchema)

In [11]:
text = "今年的利润是多少"
runnable.invoke({"text": text})

PerformanceQuerySchema(indicator=<PerformanceIndicator.GROSS_PROFIT: 3>)